# Surface Freshwater Flux / Virtual Salt Flux: E - P - R

In [ ]:
import os
import sys
sys.path.append("..")
import xarray as xr
import cmocean
import cartopy
import cartopy.crs as ccrs
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
matplotlib.rc_file('../rc_file')
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

In [ ]:
from tqdm import tqdm_notebook
from paths import path_results, path_prace, file_RMASK_ocn, file_RMASK_ocn_low, file_ex_ocn_ctrl
from timeseries import IterateOutputCESM
from xr_regression import ocn_field_regression

In [ ]:
do = xr.open_dataset(file_ex_ocn_ctrl, decode_times=False)
RMASK_ocn = do.REGION_MASK
# RMASK_ocn_low = xr.open_dataset(file_RMASK_ocn_low)

In [ ]:
# for i, (y,m,f) in tqdm_notebook(enumerate(IterateOutputCESM(domain='ocn', run='rcp', tavg='yrly', name='EVAP_F_PREC_F_ROFF_F'))):
#     ds = xr.open_dataset(f, decode_times=False)
#     if i==0:  EVAP, PREC, ROFF = [], [], []
#     EVAP.append(ds.EVAP_F)
#     PREC.append(ds.PREC_F)
#     ROFF.append(ds.ROFF_F)
# xr.concat(EVAP, dim='time').to_netcdf(f'{path_prace}/rcp/EVAP_yrly.nc')
# xr.concat(PREC, dim='time').to_netcdf(f'{path_prace}/rcp/PREC_yrly.nc')
# xr.concat(ROFF, dim='time').to_netcdf(f'{path_prace}/rcp/ROFF_yrly.nc')
EVAP = xr.open_dataarray(f'{path_prace}/rcp/EVAP_yrly.nc', decode_times=False)
PREC = xr.open_dataarray(f'{path_prace}/rcp/PREC_yrly.nc', decode_times=False)
ROFF = xr.open_dataarray(f'{path_prace}/rcp/ROFF_yrly.nc', decode_times=False)

In [ ]:
%%time
EVAP_mean = EVAP.isel(time=slice(0,30)).mean(dim='time')
PREC_mean = PREC.isel(time=slice(0,30)).mean(dim='time')
ROFF_mean = ROFF.isel(time=slice(0,30)).mean(dim='time')

In [ ]:
# %%time
# EVAP_trend = ocn_field_regression(xa=EVAP, run='rcp')
# PREC_trend = ocn_field_regression(xa=PREC, run='rcp')
# ROFF_trend = ocn_field_regression(xa=ROFF, run='rcp')
# EVAP_trend[0].to_netcdf(f'{path_prace}/rcp/EVAP_yrly_trend.nc')
# PREC_trend[0].to_netcdf(f'{path_prace}/rcp/PREC_yrly_trend.nc')
# ROFF_trend[0].to_netcdf(f'{path_prace}/rcp/ROFF_yrly_trend.nc')
EVAP_trend = xr.open_dataarray(f'{path_prace}/rcp/EVAP_yrly_trend.nc')
PREC_trend = xr.open_dataarray(f'{path_prace}/rcp/PREC_yrly_trend.nc')
ROFF_trend = xr.open_dataarray(f'{path_prace}/rcp/ROFF_yrly_trend.nc')

In [ ]:
EVAP_trend

In [ ]:
f = plt.figure(figsize=(6.4,12), constrained_layout=False)
for i, q1 in enumerate(['Precipitation', 'Evaporation', 'P+R+E']):
    mean = [PREC_mean, EVAP_mean, PREC_mean+EVAP_mean+ROFF_mean][i]
    trend = [PREC_trend, EVAP_trend, PREC_trend+EVAP_trend+ROFF_trend][i]
    mean_min, mean_max = [0,-7.5,-7.5][i], [7.5,0,7.5][i]
    trend_min, trend_max = -2.5, 2.5
    mean_cmap, trend_cmap = ['viridis', 'viridis_r', 'BrBG'][i], ['cmo.balance', 'cmo.balance_r', 'cmo.balance'][i]
    ax = f.add_subplot(3, 5, 1+i*5)
    ax.axis('off')
    ax.set_position([.04,.01+(2-i)*.32,.02,.3])
    ax.text(.5, .5, q1, transform=ax.transAxes, rotation='vertical', va='center', ha='right', fontsize=20)
    for j, q2 in tqdm_notebook(enumerate(['mean', 'trend'])):
        xa = [mean.where(RMASK_ocn>0)*24*3600, trend.where(RMASK_ocn>0)*24*3600*365*100][j]
        ax = f.add_subplot(3, 3, 3*i+2+j, projection=ccrs.LambertAzimuthalEqualArea(central_longitude=-30, central_latitude=20))
        minv, maxv, cmap = [mean_min, trend_min][j], [mean_max, trend_max][j], [mean_cmap, trend_cmap][j]
        if i==0:  ax.text(.5, 1.05, ['mean [mm/day]', 'trend [mm/day/100y]'][j], transform=ax.transAxes, fontsize=16, ha='center')
        ax.set_position([.05+j*.46,.01+(2-i)*.32,.45,.3])
        ax.set_extent((-6e6, 3.5e6, -8.5e6, 1e7), crs=ccrs.LambertAzimuthalEqualArea())
        cax, kw = matplotlib.colorbar.make_axes(ax,location='right',pad=0.01,shrink=0.9)
        im = ax.pcolormesh(PREC.ULONG, PREC.ULAT, xa, cmap=cmap,  # kg/m2/s to mm/d
#         im = ax.contourf(xa.ULONG, xa.ULAT, xa, cmap=cmap, levels=14,  # kg/m2/s to mm/d
                           vmin=minv, vmax=maxv, transform=ccrs.PlateCarree())
        cbar = f.colorbar(im, cax=cax, extend='both', **kw)
        cbar.ax.tick_params(labelsize=10)

        ax.add_feature(cartopy.feature.LAND, zorder=2, edgecolor='black', facecolor='grey')
        ax.add_feature(cartopy.feature.RIVERS, zorder=3)
        gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False)
        gl.ylocator = matplotlib.ticker.FixedLocator([-90, -60, -30, 0, 30, 60, 90])
# plt.savefig(f'{path_results}/SFWF/SFWF_map_rcp')

### time series: Arctic runoff

In [ ]:
from xr_DataArrays import xr_AREA

In [ ]:
RMASK_ocn = xr.open_dataset(file_ex_ocn_ctrl, decode_times=False).REGION_MASK

In [ ]:
RMASK_ocn

In [ ]:
Atl_MASK = xr.DataArray(np.in1d(RMASK_ocn, [6,8,9,10]).reshape(RMASK_ocn.shape),
             dims=RMASK_ocn.dims, coords=RMASK_ocn.coords)

In [ ]:
Atl_MASK.plot()

In [ ]:
AREA_ocn = xr_AREA(domain='ocn')

In [ ]:
for (latS, latN) in [(45,70),(10,45),(-10,10), (-34,10), (-34,70), (70,90)]:
    AREA_total = AREA_ocn .where(Atl_MASK).where(PREC_mean.ULAT<latN).where(PREC_mean.ULAT>latS).sum()
    PREC_AREA = (PREC_mean.where(Atl_MASK).where(PREC_mean.ULAT<latN).where(PREC_mean.ULAT>latS)*AREA_ocn).sum()
    EVAP_AREA = (EVAP_mean.where(Atl_MASK).where(PREC_mean.ULAT<latN).where(PREC_mean.ULAT>latS)*AREA_ocn).sum()
    ROFF_AREA = (ROFF_mean.where(Atl_MASK).where(PREC_mean.ULAT<latN).where(PREC_mean.ULAT>latS)*AREA_ocn).sum()
    
    PREC_trend_AREA = (PREC_trend.where(Atl_MASK).where(PREC_mean.ULAT<latN).where(PREC_mean.ULAT>latS)*AREA_ocn).sum()
    EVAP_trend_AREA = (EVAP_trend.where(Atl_MASK).where(PREC_mean.ULAT<latN).where(PREC_mean.ULAT>latS)*AREA_ocn).sum()
    ROFF_trend_AREA = (ROFF_trend.where(Atl_MASK).where(PREC_mean.ULAT<latN).where(PREC_mean.ULAT>latS)*AREA_ocn).sum()
    print(f'\n{latS}N to {latN}N,   {AREA_total.values:4.2E} m^2\n')
    print('             PREC  EVAP  ROFF    SUM')

    print(f'[mm/d]      {(PREC_AREA/AREA_total).values*24*3600:5.2f}',
          f'{(EVAP_AREA/AREA_total).values*24*3600:5.2f}',
          f'{(ROFF_AREA/AREA_total).values*24*3600:5.2f}',
          f'{((PREC_AREA+EVAP_AREA+ROFF_AREA)/AREA_total).values*24*3600:5.4f}')
    print(f'[mm/d/100y] {(PREC_trend_AREA/AREA_total).values*24*3600*365*100:5.2f}',
          f'{(EVAP_trend_AREA/AREA_total).values*24*3600*365*100:5.2f}',
          f'{(ROFF_trend_AREA/AREA_total).values*24*3600*365*100:5.2f}',
          f'{((PREC_trend_AREA+EVAP_trend_AREA+ROFF_trend_AREA)/AREA_total).values*24*3600*365*100:5.4f}')
    
    print(f'[Sv]        {PREC_AREA.values/1e9:5.2f}',  # kg/m^2/s -> Sv
          f'{EVAP_AREA.values/1e9:5.2f}',
          f'{ROFF_AREA.values/1e9:5.2f}',
          f'{(PREC_AREA+EVAP_AREA+ROFF_AREA).values/1e9:5.4f}')
    print(f'[Sv/100y]   {PREC_trend_AREA.values/1e9*365*100:5.2f}',
          f'{EVAP_trend_AREA.values/1e9*365*100:5.2f}',
          f'{ROFF_trend_AREA.values/1e9*365*100:5.2f}',
          f'{(PREC_trend_AREA+EVAP_trend_AREA+ROFF_trend_AREA).values/1e9*365*100:5.4f}')
    
    print(f'[%/100y]    {(PREC_trend_AREA.values/1e9*365*100)/(PREC_AREA.values/1e9)*100:5.1f}',
          f'{(EVAP_trend_AREA.values*365*100)/(EVAP_AREA.values)*100:5.1f}',
          f'{(ROFF_trend_AREA.values*365*100)/(ROFF_AREA.values)*100:5.1f}',
          f'{((PREC_trend_AREA+EVAP_trend_AREA+ROFF_trend_AREA).values*365*100/((PREC_AREA+EVAP_AREA+ROFF_AREA).values))*100:7.1f}')